<table class="tfo-notebook-buttons" align="left">

  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rajdeepd/tensorflow_2.0_book_code/blob/master/ch09/cqat_example_mnist"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/rajdeepd/tensorflow_2.0_book_code/blob/master/ch09/cqat_example_mnist.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>

</table>

# Cluster preserving quantization aware training (CQAT) Keras example

## Overview

This is an end to end example showing the usage of the **cluster preserving quantization aware training (CQAT)** API, part of the TensorFlow Model Optimization Toolkit's collaborative optimization pipeline.

### Other pages

For an introduction to the pipeline and other available techniques, see the [collaborative optimization overview page](https://www.tensorflow.org/model_optimization/guide/combine/collaborative_optimization).

### Contents

In the tutorial, you will:

1. Train a `tf.keras` model for the MNIST dataset from scratch.
2. Fine-tune the model with clustering and see the accuracy.
3. Apply QAT and observe the loss of clusters.
4. Apply CQAT and observe that the clustering applied earlier has been preserved.
5. Generate a TFLite model and observe the effects of applying CQAT on it.
6. Compare the achieved CQAT model accuracy with a model quantized using post-training quantization.

## Setup

You can run this Jupyter Notebook in local [virtualenv](https://www.tensorflow.org/install/pip?lang=python3#2.-create-a-virtual-environment-recommended) or [colab](https://colab.sandbox.google.com/).

In [1]:
! pip install -q tensorflow-model-optimization

     |████████████████████████████████| 213 kB 32.2 MB/s 


## Make the necessary import

In [2]:
import tensorflow as tf

import numpy as np
import tempfile
import zipfile
import os

## Train a tf.keras model for MNIST without clustering
1. load the dataset
2. train and test images normalize
3. Create Sequential model
4. Compile the model with following parameters
  
   * Use `adam` optimizer
   * `SparseCategoricalCrossentropy`
   * Optimize for `accuracy` metrics
5. Run model.fit(..) with `train_images` and `train_labels` for 10 epochs and validation split of 0.1

In [3]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images  = test_images / 255.0

model = tf.keras.Sequential([
  tf.keras.layers.InputLayer(input_shape=(28, 28)),
  tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
  tf.keras.layers.Conv2D(filters=12, kernel_size=(3, 3),
                         activation=tf.nn.relu),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
    train_images,
    train_labels,
    validation_split=0.1,
    epochs=10
)

11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.3037 - accuracy: 0.9137 - val_loss: 0.1267 - val_accuracy: 0.9680
Epoch 2/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1228 - accuracy: 0.9649 - val_loss: 0.0912 - val_accuracy: 0.9743
Epoch 3/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0863 - accuracy: 0.9751 - val_loss: 0.0721 - val_accuracy: 0.9800
Epoch 4/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0696 - accuracy: 0.9799 - val_loss: 0.0637 - val_accuracy: 0.9825
Epoch 5/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0593 - accuracy: 0.9823 - val_loss: 0.0717 - val_accuracy: 0.9802
Epoch 6/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0527 - accuracy: 0.9845 - val_loss: 0.0590 - val_accuracy: 0.9843
Epoch 7/10
1688/1688 [==============================] - 17s

### Evaluate the baseline model and save it for later usage

In [4]:
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Baseline test accuracy: 0.9824000000953674
Saving model to:  /tmp/tmp6feo6no6.h5


## Cluster and fine-tune the model with 8 clusters

Apply the `cluster_weights()` API to cluster the whole pre-trained model to demonstrate and observe its effectiveness in reducing the model size when applying zip, while maintaining accuracy. For more details refer to the  [clustering comprehensive guide](https://www.tensorflow.org/model_optimization/guide/clustering/clustering_comprehensive_guide).

### Define the model and apply the clustering API

The model needs to be pre-trained before using the clustering API. This function wraps a keras model or layer with clustering functionality which clusters the layer's weights during training. For examples, using this with number_of_clusters equals 8 will ensure that each weight tensor has no more than 8 unique values.

In [5]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 8,
  'cluster_centroids_init': CentroidInitialization.KMEANS_PLUS_PLUS
}

clustered_model = cluster_weights(model, **clustering_params)

# Use smaller learning rate for fine-tuning
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cluster_reshape (ClusterWeig (None, 28, 28, 1)         0         
_________________________________________________________________
cluster_conv2d (ClusterWeigh (None, 26, 26, 12)        236       
_________________________________________________________________
cluster_max_pooling2d (Clust (None, 13, 13, 12)        0         
_________________________________________________________________
cluster_flatten (ClusterWeig (None, 2028)              0         
_________________________________________________________________
cluster_dense (ClusterWeight (None, 10)                40578     
Total params: 40,814
Trainable params: 20,426
Non-trainable params: 20,388
_________________________________________________________________


### Fine-tune the model and evaluate the accuracy against baseline

Fine-tune the model with clustering for 3 epochs.

In [6]:
# Fine-tune model
clustered_model.fit(
  train_images,
  train_labels,
  epochs=3,
  validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 22s 13ms/step - loss: 0.0494 - accuracy: 0.9843 - val_loss: 0.0648 - val_accuracy: 0.9837
Epoch 2/3
1688/1688 [==============================] - 21s 13ms/step - loss: 0.0361 - accuracy: 0.9887 - val_loss: 0.0614 - val_accuracy: 0.9843
Epoch 3/3
1688/1688 [==============================] - 21s 12ms/step - loss: 0.0340 - accuracy: 0.9895 - val_loss: 0.0595 - val_accuracy: 0.9848


Define helper functions to calculate and print the number of clustering in each kernel of the model.

In [7]:
def print_model_weight_clusters(model):

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights
        for weight in weights:
            # ignore auxiliary quantization weights
            if "quantize_layer" in weight.name:
                continue
            if "kernel" in weight.name:
                unique_count = len(np.unique(weight))
                print(
                    f"{layer.name}/{weight.name}: {unique_count} clusters "
                )

Check that the model kernels were correctly clustered. We need to strip the clustering wrapper first.

In [8]:
stripped_clustered_model = tfmot.clustering.keras.strip_clustering(clustered_model)

print_model_weight_clusters(stripped_clustered_model)

conv2d/kernel:0: 8 clusters 
dense/kernel:0: 8 clusters 


For this example, there is minimal loss in test accuracy after clustering, compared to the baseline.

In [9]:
_, clustered_model_accuracy = clustered_model.evaluate(
  test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Clustered test accuracy:', clustered_model_accuracy)

Baseline test accuracy: 0.9824000000953674
Clustered test accuracy: 0.9797000288963318


## Apply QAT and CQAT and check effect on model clusters in both cases

Next, we apply both QAT and cluster preserving QAT (CQAT) on the clustered model and observe that CQAT preserves weight clusters in your clustered model. Note that we stripped clustering wrappers from your model with `tfmot.clustering.keras.strip_clustering` before applying CQAT API.

In [10]:
# QAT
qat_model = tfmot.quantization.keras.quantize_model(stripped_clustered_model)

qat_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
print('Train qat model:')
qat_model.fit(train_images, train_labels, batch_size=128, epochs=1, validation_split=0.1)

# CQAT
quant_aware_annotate_model = tfmot.quantization.keras.quantize_annotate_model(
              stripped_clustered_model)
cqat_model = tfmot.quantization.keras.quantize_apply(
              quant_aware_annotate_model,
              tfmot.experimental.combine.Default8BitClusterPreserveQuantizeScheme())

cqat_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
print('Train cqat model:')
cqat_model.fit(train_images, train_labels, batch_size=128, epochs=1, validation_split=0.1)

Train qat model:
422/422 [==============================] - 17s 38ms/step - loss: 0.0332 - accuracy: 0.9904 - val_loss: 0.0564 - val_accuracy: 0.9858


Train cqat model:


422/422 [==============================] - 17s 39ms/step - loss: 0.0315 - accuracy: 0.9907 - val_loss: 0.0581 - val_accuracy: 0.9853


In [11]:
print("QAT Model clusters:")
print_model_weight_clusters(qat_model)
print("CQAT Model clusters:")
print_model_weight_clusters(cqat_model)

QAT Model clusters:
quant_conv2d/conv2d/kernel:0: 108 clusters 
quant_dense/dense/kernel:0: 19867 clusters 
CQAT Model clusters:
quant_conv2d/conv2d/kernel:0: 8 clusters 
quant_dense/dense/kernel:0: 8 clusters 


## See compression benefits of CQAT model

Define helper function to get zipped model file.

In [12]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in kilobytes.

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)/1000

Note that this is a small model. Applying clustering and CQAT to a bigger production model would yield a more significant compression.

In [13]:
# QAT model
converter = tf.lite.TFLiteConverter.from_keras_model(qat_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
qat_tflite_model = converter.convert()
qat_model_file = 'qat_model.tflite'
# Save the model.
with open(qat_model_file, 'wb') as f:
    f.write(qat_tflite_model)
    
# CQAT model
converter = tf.lite.TFLiteConverter.from_keras_model(cqat_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
cqat_tflite_model = converter.convert()
cqat_model_file = 'cqat_model.tflite'
# Save the model.
with open(cqat_model_file, 'wb') as f:
    f.write(cqat_tflite_model)
    
print("QAT model size: ", get_gzipped_model_size(qat_model_file), ' KB')
print("CQAT model size: ", get_gzipped_model_size(cqat_model_file), ' KB')

INFO:tensorflow:Assets written to: /tmp/tmp0m0bw5gt/assets


INFO:tensorflow:Assets written to: /tmp/tmp0m0bw5gt/assets


INFO:tensorflow:Assets written to: /tmp/tmpcsap1kjc/assets


INFO:tensorflow:Assets written to: /tmp/tmpcsap1kjc/assets


QAT model size:  16.32  KB
CQAT model size:  10.179  KB


## See the persistence of accuracy from TF to TFLite

Define a helper function to evaluate the TFLite model on the test dataset.

In [14]:
def eval_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(test_images):
    if i % 1000 == 0:
      print(f"Evaluated on {i} results so far.")
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == test_labels).mean()
  return accuracy

You evaluate the model, which has been clustered and quantized, and then see the accuracy from TensorFlow persists in the TFLite backend.

In [15]:
interpreter = tf.lite.Interpreter(cqat_model_file)
interpreter.allocate_tensors()

cqat_test_accuracy = eval_model(interpreter)

print('Clustered and quantized TFLite test_accuracy:', cqat_test_accuracy)
print('Clustered TF test accuracy:', clustered_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Clustered and quantized TFLite test_accuracy: 0.982
Clustered TF test accuracy: 0.9797000288963318


## Apply post-training quantization and compare to CQAT model

Next, we use post-training quantization (no fine-tuning) on the clustered model and check its accuracy against the CQAT model. This demonstrates why you would need to use CQAT to improve the quantized model's accuracy.

First, define a generator for the callibration dataset from the first 1000 training images.

In [16]:
def mnist_representative_data_gen():
  for image in train_images[:1000]:  
    image = np.expand_dims(image, axis=0).astype(np.float32)
    yield [image]

Quantize the model and compare accuracy to the previously acquired CQAT model. Note that the model quantized with fine-tuning achieves higher accuracy.

In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(stripped_clustered_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = mnist_representative_data_gen
post_training_tflite_model = converter.convert()
post_training_model_file = 'post_training_model.tflite'
# Save the model.
with open(post_training_model_file, 'wb') as f:
    f.write(post_training_tflite_model)
    
# Compare accuracy
interpreter = tf.lite.Interpreter(post_training_model_file)
interpreter.allocate_tensors()

post_training_test_accuracy = eval_model(interpreter)

print('CQAT TFLite test_accuracy:', cqat_test_accuracy)
print('Post-training (no fine-tuning) TF test accuracy:', post_training_test_accuracy)

INFO:tensorflow:Assets written to: /tmp/tmp29vbdicc/assets


INFO:tensorflow:Assets written to: /tmp/tmp29vbdicc/assets


Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


CQAT TFLite test_accuracy: 0.982
Post-training (no fine-tuning) TF test accuracy: 0.9795


## Conclusion

In this tutorial, you learned how to create a model, cluster it using the `cluster_weights()` API, and apply the cluster preserving quantization aware training (CQAT) to preserve clusters while using QAT. The final CQAT model was compared to the QAT one to show that the clusters are preserved in the former and lost in the latter. Next, the models were converted to TFLite to show the compression benefits of chaining clustering and CQAT model optimization techniques and the TFLite model was evaluated to ensure that the accuracy persists in the TFLite backend. Finally, the CQAT model was compared to a quantized clustered model achieved using the post-training quantization API to demonstrate the advantage of CQAT in recovering the accuracy loss from normal quantization.